In [1]:
require(ChIPseeker)
library(rGREAT)
require(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(ggplot2)
library(dplyr)

载入需要的程序包：ChIPseeker



ChIPseeker v1.42.1 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

Guangchuang Yu, Li-Gen Wang, and Qing-Yu He. ChIPseeker: an
R/Bioconductor package for ChIP peak annotation, comparison and
visualization. Bioinformatics. 2015, 31(14):2382-2383

载入需要的程序包：GenomicRanges

载入需要的程序包：stats4

载入需要的程序包：BiocGenerics


载入程序包：‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min


载入需要的程序包：S4Vectors


载入程序包：‘S4Vectors’


The following object i

In [2]:
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

In [3]:
#peaks <- read.table("/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_enrichment/DAR_unique_peaks.bed", header = FALSE, col.names = c("chr", "start", "end"))
peaks <- data.frame(
  chr   = c("chr16", "chr2"),
  start = c(3065087, 136132091),
  end   = c(3065588, 136132592),
  stringsAsFactors = FALSE
)
gr <- makeGRangesFromDataFrame(peaks, ignore.strand = TRUE)
peakAnno <- annotatePeak(gr, tssRegion=c(-20000, 20000), 
                         TxDb=txdb, annoDb="org.Hs.eg.db")
annotation <- peakAnno@annoStat

>> preparing features information...		 2026-01-04 14时11分57秒 
>> identifying nearest features...		 2026-01-04 14时11分59秒 
>> calculating distance from peak to TSS...	 2026-01-04 14时12分01秒 
>> assigning genomic annotation...		 2026-01-04 14时12分01秒 
>> adding gene annotation...			 2026-01-04 14时12分37秒 


载入需要的程序包：org.Hs.eg.db

'select()' returned 1:1 mapping between keys and columns



>> assigning chromosome lengths			 2026-01-04 14时12分37秒 
>> done...					 2026-01-04 14时12分37秒 


In [4]:
annotation

,Feature,Frequency
,<fct>,<dbl>
1,Promoter (<=1kb),50
2,Promoter (12-13kb),50


In [5]:
bed_dir <- "/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_bed/"
output_csv <- "/media/AnalysisDisk1/xuzekai/20250715_BMI/ATAC/DAR_enrichment/all_celltypes_annotation_summary.csv"

# 加载数据库（根据你的物种和参考基因组）
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene  # 或 TxDb.Hsapiens.UCSC.hg19.knownGene 等
# 如果报错找不到，可以用：txdb <- loadDb("your_txdb.sqlite")

# 获取所有 .bed 文件
bed_files <- list.files(bed_dir, pattern = "*.bed$", full.names = TRUE)
names(bed_files) <- gsub("\\.bed$", "", basename(bed_files))  # 去掉 .bed 作为 celltype 名称

# 存储结果的列表
results_list <- list()

# 循环处理每个细胞类型
for (file in bed_files) {
  celltype <- gsub("\\.bed$", "", basename(file))
  message("Processing: ", celltype)
  
  # 读取 peaks
  peaks <- read.table(file, header = FALSE, col.names = c("chr", "start", "end"))
  
  # 转为 GRanges
  gr <- makeGRangesFromDataFrame(peaks, ignore.strand = TRUE)
  
  # 注释 peaks
  peakAnno <- annotatePeak(gr, 
                           tssRegion = c(-3000, 3000), 
                           TxDb = txdb, 
                           annoDb = "org.Hs.eg.db")
  
  # 提取 annoStat（统计表）
  annotation_df <- peakAnno@annoStat
  
  # 添加 celltype 列
  annotation_df$celltype <- celltype
  library(ggplot2)
library(dplyr)

# 确保 Feature 是因子，并设置你喜欢的顺序
all_results <- all_results %>%
  mutate(Feature = factor(Feature, 
                          levels = rev(c("Promoter (<=1kb)", "Promoter (1-2kb)", "Promoter (2-3kb)",
                                         "5' UTR", "3' UTR", "1st Exon", "Other Exon",
                                         "1st Intron", "Other Intron",
                                         "Downstream (<=300)", "Distal Intergenic"))))

# 绘制堆叠柱状图：每个细胞类型一个柱子
p <- ggplot(all_results, aes(x = celltype, y = Frequency, fill = Feature)) +
  geom_col() +  # 堆叠柱状图
  scale_y_continuous(expand = expansion(mult = c(0, 0.05))) +
  scale_fill_brewer(type = "qual", palette = "Set3", direction = -1) +  # reverse colors
  labs(
    title = "Genomic Feature Distribution by Cell Type",
    x = "Cell Type",
    y = "Percentage (%)"
  ) +
  theme_bw() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10),
    legend.title = element_text(size = 11),
    legend.text = element_text(size = 9),
    legend.position = "right"
  )

# 显示图形
print(p)
  # 存入列表
  results_list[[celltype]] <- annotation_df
}

# 合并所有结果
all_results <- do.call(rbind, results_list)

# 重置行名
rownames(all_results) <- NULL

# 查看前几行
print(head(all_results))

# 保存为 CSV
write.csv(all_results, output_csv, row.names = FALSE)

message("✅ 所有细胞类型的注释汇总已保存至: ", output_csv)


Processing: Atypical_Bm_ITGAX



>> preparing features information...		 2026-01-04 14时12分38秒 
>> identifying nearest features...		 2026-01-04 14时12分38秒 
>> calculating distance from peak to TSS...	 2026-01-04 14时12分38秒 
>> assigning genomic annotation...		 2026-01-04 14时12分38秒 
>> adding gene annotation...			 2026-01-04 14时12分42秒 


'select()' returned 1:many mapping between keys and columns



>> assigning chromosome lengths			 2026-01-04 14时12分42秒 
>> done...					 2026-01-04 14时12分42秒 


ERROR: Error: 找不到对象'all_results'


In [ ]:
epeak_number_final_cs_ch<-read.csv('/media/AnalysisDisk1/xuzekai/20250115_CIMA/2025_0212_CIMA_eGene_caPeak_Statistics/eGene_cellspecial_number_20250117.csv')
celltype_mapping <- tibble::tribble(
  ~celltype,     ~celltype.1,
    "AS_DC",       "Mono",
    "CD4_Tem_CCR5",       "CD4",
    "CD4_Tr1_IL10",       "CD4",
    "DC1_CLEC9A",       "Mono",
    "DC2_CD1C",       "Mono",
    "DC_CSF2RA",       "Mono",
    "Immature_T_CABP4",       "CD8",
    "Proliferative_NK_MKI67",       "NK",
    "Proliferative_T_MKI67",       "CD8",
    "Transitional_B_MME",       "B",
    "pre-Switched_Bm_IFIT3",       "B",
  # 可以在这里继续添加其他映射
)

# 如果你还想保留 epeak_number_final_cs_ch 中的其他映射，可以合并
# 假设原表 epeak_number_final_cs_ch 有部分映射，但不完整
full_mapping <- bind_rows(
  celltype_mapping,
  epeak_number_final_cs_ch %>% select(celltype, celltype.1)
) %>% distinct(celltype, .keep_all = TRUE)

In [ ]:
all_results

In [ ]:
result_wide_mapped <- all_results %>%
  left_join(full_mapping, by = "celltype")
# 查看结果
#result_wide_mapped= result_wide_mapped%>%select(celltype,down,up,celltype.1)
result_wide_mapped

In [ ]:
super_group_order <- c("B", "CD4", "CD8", "NK", "Mono")

# 创建排序键
ordering_table <- result_wide_mapped  %>%
  mutate(
   celltype.1 = factor(celltype.1, levels = super_group_order),
    celltype = factor(celltype, levels = unique(celltype))  # 可自定义顺序
  ) %>%
  arrange(celltype.1, celltype)  # 按大类 + 名称排序

In [ ]:
ordering_table

In [ ]:
unique_celltype_order <- unique(ordering_table$celltype)

# 2. 将 all_results_with_group 中的 celltype 设为有序因子
ordering_table$celltype <- factor(
  ordering_table$celltype,
  levels = unique_celltype_order,    # ← 这里传入唯一值的向量
  ordered = TRUE
)

In [ ]:
# 确保 Feature 是因子，并设置你喜欢的顺序
ordering_table <- ordering_table %>%
  mutate(Feature = factor(Feature, 
                          levels = rev(c("Promoter (<=1kb)", "Promoter (1-2kb)", "Promoter (2-3kb)",
                                         "5' UTR", "3' UTR", "1st Exon", "Other Exon",
                                         "1st Intron", "Other Intron",
                                         "Downstream (<=300)", "Distal Intergenic"))))

# 绘制堆叠柱状图：每个细胞类型一个柱子
p <- ggplot(ordering_table, aes(x = celltype, y = Frequency, fill = Feature)) +
  geom_col() +  # 堆叠柱状图
  scale_y_continuous(expand = expansion(mult = 0)) +
  scale_fill_brewer(type = "qual", palette = "Set3", direction = -1) +  # reverse colors
  labs(
    title = "Genomic Feature Distribution by Cell Type",
    x = "Cell Type",
    y = "Percentage (%)"
  ) +
  theme_bw() +
  theme(
    axis.text.x = element_text(angle = 90, hjust = 1, size = 10),
    legend.title = element_text(size = 11),
    legend.text = element_text(size = 9),
    legend.position = "right"
  )

# 显示图形
print(p)

In [ ]:
ggsave("feature_distribution_by_celltype.pdf", p, width = 10, height = 6)

In [ ]:
plotAnnoBar.data.frame <- function(anno.df,
                                   xlab="",
                                   ylab="Percentage(%)",
                                   title="Feature Distribution",
                                   categoryColumn) {


    anno.df$Feature <- factor(anno.df$Feature, levels = rev(levels(anno.df$Feature)))

    p <- ggplot(anno.df, aes_string(x = categoryColumn,
                                    fill = "Feature",
                                    y = "Frequency"))

    p <- p + geom_bar(stat="identity") + coord_flip() + theme_bw()
    p <- p + ylab(ylab) + xlab(xlab) + ggtitle(title)

    if (categoryColumn == 1) {
        p <- p + scale_x_continuous(breaks=NULL)
        p <- p+scale_fill_manual(values=rev(getCols(nrow(anno.df))), guide=guide_legend(reverse=TRUE))
    } else {
        p <- p+scale_fill_manual(values=rev(getCols(length(unique(anno.df$Feature)))), guide=guide_legend(reverse=TRUE))
    }

    return(p)
}

In [ ]:
pdf("peak_annotation_pie.pdf", width = 8, height = 6)
plotAnnoPie(peakAnno)
dev.off()

In [ ]:
plotAnnoBar(peakAnno)

In [ ]:
peaks <- GRanges(
  seqnames = "chr15",
  ranges = IRanges(start = 39580700, end = 39581201),
  name = "Peak_1"  # 可选：给 peak 起个名字
)

In [ ]:
peakAnno <- annotatePeak(peaks, tssRegion=c(-3000, 3000), 
                         TxDb=txdb, annoDb="org.Hs.eg.db")

In [ ]:
peakAnno